# Swarm Behavior Cloning: Multi-Agent Coordination Example

## Overview

This code implements a Swarm Behavior Cloning (Swarm BC) system for a travel planning workflow. It demonstrates how multiple agents (e.g., Weather Expert, Flight Advisor) are trained and coordinated to provide a cohesive travel plan.

The workflow uses embeddings, neural networks, and GPT-4o for text generation and aggregation. Here’s the annotated code with explanations.

### Import Libraries and Setup

In [1]:
#!pip install openai numpy sentence-transformers tf-keras tensorflow keras

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from openai import OpenAI
import os
import logging
from sentence_transformers import SentenceTransformer
import json

/Users/anna/Projects/PolyAgentOps/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### API Key Setup

In [ ]:
# Set up OpenAI API key (replace with your own key)
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'

### Define the Embeddings Model and Syntax to call Open Ai

In [4]:
client = OpenAI()

def call_openai_chat(agent_name, system_prompt, user_query, client):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"You are {agent_name}, an expert in {system_prompt}."},
            {"role": "user", "content": user_query}
        ]
    )
    print(completion.choices[0].message.content)

    return completion.choices[0].message.content


In [5]:
# Initialize embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

### Define Agent Tasks and Initialization

In [6]:
# Define task-specific travel agents
agents_info = {
    "Weather Expert": "providing weather forecasts and climate guidance",
    "Flight Advisor": "finding optimal flights and itineraries",
    "Budget Planner": "creating a comprehensive travel budget",
    "Sightseeing Guide": "suggesting cultural and historical attractions",
    "Accommodation Expert": "booking the best hotels and stays",
    "Dining Consultant": "recommending top restaurants and dining experiences"
}

### Neural Network Model for Agents

In [7]:
# Create a neural network model for agents
def create_agent(input_dim, output_dim):
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dense(output_dim, activation='sigmoid')  # Ensure output is between 0 and 1
    ])
    return model

### Initialize Agents and Optimizers

In [8]:
# Initialize agents
num_agents = len(agents_info)
input_dim = 384  # Sentence Transformer output size
output_dim = 4  # Output prediction vector

In [9]:
# Define models and optimizers
agents = {task: create_agent(input_dim, output_dim) for task in agents_info}
optimizers = {task: tf.keras.optimizers.Adam(learning_rate=0.0005) for task in agents_info}


/Users/anna/Projects/PolyAgentOps/venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Define the Swarm BC loss function
loss_fn = tf.keras.losses.MeanSquaredError()

### Swarm BC Loss Function

In [11]:

def swarm_bc_loss(predictions, targets, agents_outputs):
    prediction_loss = loss_fn(predictions, targets)
    regularization_loss = sum(
        tf.reduce_mean(tf.square(a_i - a_j))
        for i, a_i in enumerate(agents_outputs)
        for j, a_j in enumerate(agents_outputs) if i < j
    )
    return prediction_loss + 0.1 * regularization_loss


### Convert Text to Embeddings & Generate Synthetic Travel Scenarios

In [12]:
# Convert text responses to embeddings
def map_response_to_embedding(response_text):
    return embedding_model.encode(response_text)

# Generate synthetic travel scenarios
def generate_synthetic_states(num_samples):
    destinations = ["Kyoto", "Paris", "Rome", "Barcelona", "New York"]
    seasons = ["Spring", "Summer", "Autumn", "Winter"]
    budgets = [1000, 2000, 3000, 4000, 5000]
    interests = ["history", "art", "food", "nature", "adventure"]

    states = []
    targets = []

    for _ in range(num_samples):
        destination = np.random.choice(destinations)
        season = np.random.choice(seasons)
        budget = np.random.choice(budgets)
        interest1 = np.random.choice(interests)
        interest2 = np.random.choice(interests)
        days = np.random.randint(3, 15)

        trip_description = (
            f"Plan a trip to {destination} in {season} with a budget of {budget} USD for {days} days. "
            f"Focus on {interest1} and {interest2}."
        )

        state_vector = map_response_to_embedding(trip_description)
        target_vector = np.array([budget / 5000, days / 15, np.random.rand(), np.random.rand()])

        states.append(state_vector)
        targets.append(target_vector)

    return np.array(states), np.array(targets)

### Aggregation Prompt Construction

In [13]:

# Construct the dynamic aggregation prompt
def construct_prompt(original_query, agent_responses, aggregated_predictions):
    scaled = {
        "Budget Fit": aggregated_predictions[0],
        "Duration Suitability": aggregated_predictions[1],
        "Cultural Relevance": aggregated_predictions[2],
        "Comfort Level": aggregated_predictions[3]
    }

    responses_formatted = json.dumps(agent_responses, indent=2)

    return f"""
    Original Query: "{original_query}"

    Agent Responses:
    {responses_formatted}

    Aggregated Predictions (Scaled 0-1):
    - Budget Fit: {scaled['Budget Fit']:.2f}
    - Duration Suitability: {scaled['Duration Suitability']:.2f}
    - Cultural Relevance: {scaled['Cultural Relevance']:.2f}
    - Comfort Level: {scaled['Comfort Level']:.2f}

    Generate a comprehensive plan that addresses these points and refine the travel plan accordingly.
    """


### Training and Inference

In [14]:

# Training loop
num_epochs = 1000
state, true_action = generate_synthetic_states(num_agents)

for epoch in range(num_epochs):
    agents_outputs = []
    with tf.GradientTape(persistent=True) as tape:
        for task, agent in agents.items():
            agents_output = agent(state)
            agents_outputs.append(agents_output)
        avg_prediction = tf.reduce_mean(agents_outputs, axis=0)
        loss = swarm_bc_loss(avg_prediction, true_action, agents_outputs)

    for task, agent in agents.items():
        gradients = tape.gradient(loss, agent.trainable_variables)
        optimizers[task].apply_gradients(zip(gradients, agent.trainable_variables))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.numpy():.4f}")

# Testing with API calls
new_query = "Plan a luxury trip to Venice with a focus on art, history, and fine dining."
agent_responses = {}
raw_outputs = {}

Epoch 0: Loss = 0.0605
Epoch 100: Loss = 0.0000
Epoch 200: Loss = 0.0000
Epoch 300: Loss = 0.0000
Epoch 400: Loss = 0.0000
Epoch 500: Loss = 0.0000
Epoch 600: Loss = 0.0000
Epoch 700: Loss = 0.0000
Epoch 800: Loss = 0.0000
Epoch 900: Loss = 0.0000


### Final Plan

In [15]:
for agent_name, task_description in agents_info.items():
    agent_response = call_openai_chat(agent_name, task_description, new_query, client)
    agent_embedding = map_response_to_embedding(agent_response)
    agent_model = agents[agent_name]
    raw_output = agent_model(agent_embedding.reshape(1, -1)).numpy().flatten().tolist()
    agent_responses[agent_name] = {
        "response": agent_response,
        "raw_output": raw_output
    }
    raw_outputs[agent_name] = raw_output

Planning a luxury trip to Venice with an emphasis on art, history, and fine dining can truly be an unforgettable experience. Here's a suggested itinerary that covers these key interests:

### Day 1: Arrival and Luxury Accommodation
**Morning:**
- **Accommodation**: Check into the **Aman Venice**, a stunning hotel located in the Palazzo Papadopoli. It offers opulent rooms and suites with views of the Grand Canal.

**Afternoon:**
- **Relaxation**: Spend some time at the hotel spa to unwind after your journey.

**Evening:**
- **Dinner**: Dine at **Ristorante Quadri**, a Michelin-starred restaurant overlooking Piazza San Marco, where you can enjoy exquisite Venetian cuisine and a fine selection of wines.

### Day 2: Art and Architecture
**Morning:**
- **Art Tour**: Start your day at the **Gallerie dell'Accademia**, home to masterpieces by Bellini, Canaletto, and Titian. Opt for a private guided tour for a personalized experience.

**Afternoon:**
- **Shopping and Exploration**: Walk along t

In [16]:
# Weighted Aggregation
weights = {
    "Weather Expert": np.array([0.2, 0.3, 0.1, 0.4]),
    "Flight Advisor": np.array([0.4, 0.3, 0.2, 0.1]),
    "Budget Planner": np.array([0.5, 0.2, 0.1, 0.2]),
    "Sightseeing Guide": np.array([0.2, 0.4, 0.3, 0.1]),
    "Accommodation Expert": np.array([0.3, 0.2, 0.1, 0.4]),
    "Dining Consultant": np.array([0.1, 0.2, 0.5, 0.2])
}

In [17]:

aggregated_prediction = np.zeros(4)
total_weight = np.zeros(4)

for task, prediction in raw_outputs.items():
    weight = weights[task]
    aggregated_prediction += np.array(prediction) * weight
    total_weight += weight

aggregated_prediction /= total_weight

In [18]:
aggregated_prediction

array([0.48779808, 0.7430285 , 0.8583613 , 0.47066119])

In [19]:
# Generate Final Plan Using GPT-4o
final_prompt = construct_prompt(new_query, agent_responses, aggregated_prediction)



In [20]:
print(final_prompt)


    Original Query: "Plan a luxury trip to Venice with a focus on art, history, and fine dining."

    Agent Responses:
    {
  "Weather Expert": {
    "response": "Planning a luxury trip to Venice with an emphasis on art, history, and fine dining can truly be an unforgettable experience. Here's a suggested itinerary that covers these key interests:\n\n### Day 1: Arrival and Luxury Accommodation\n**Morning:**\n- **Accommodation**: Check into the **Aman Venice**, a stunning hotel located in the Palazzo Papadopoli. It offers opulent rooms and suites with views of the Grand Canal.\n\n**Afternoon:**\n- **Relaxation**: Spend some time at the hotel spa to unwind after your journey.\n\n**Evening:**\n- **Dinner**: Dine at **Ristorante Quadri**, a Michelin-starred restaurant overlooking Piazza San Marco, where you can enjoy exquisite Venetian cuisine and a fine selection of wines.\n\n### Day 2: Art and Architecture\n**Morning:**\n- **Art Tour**: Start your day at the **Gallerie dell'Accademia*

In [21]:
final_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert travel planner."},
            {"role": "user", "content": final_prompt}
        ]
    )


In [22]:
print(final_response.choices[0].message.content)

Planning a luxury trip to Venice with a focus on art, history, and fine dining ensures a captivating and enriching experience. Here's a refined comprehensive plan considering each aspect of your luxurious journey:

### Duration and Overview:
- **Total Duration**: 5 days and 4 nights
- **Timing**: Late spring or early autumn, when the weather is pleasant, and tourist crowds are less intense.
- **Accommodation**: Luxury, comfort, and cultural relevance are priorities.
- **Budget**: High-end; recommendations will consider premium options.

### Day 1: Arrival and Venetian Introduction
- **Accommodation**: Check into **Aman Venice** for unparalleled luxury and historical charm, or **The Gritti Palace** for a mix of opulence and tradition.
- **Morning**: Arrive at Marco Polo Airport. Transfer via private water taxi to your hotel for a grand introduction to the city.
- **Afternoon**: Visit the **Peggy Guggenheim Collection** to see iconic modern artworks in a palatial setting.
- **Evening**: 